# Question for this notebook- Find various SKU's that have seasonal purchasing patterns, try to identify reasons for seasonal changes.

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [4]:
baskets = pd.read_csv('baskets_sample_random_10.csv')

#### Implement Dr. Li's date configuration for ease of time access.

In [6]:
baskets['datetime'] = baskets['placed_at'].apply(lambda x: datetime.fromisoformat(x))

#pandas.Series.dt is an interface on a pandas series that gives you convenient access to operations on data stored as a pandas datetime. 
baskets['date'] = baskets['datetime'].dt.date
baskets['year'] = baskets['datetime'].dt.year
baskets['month'] = baskets['datetime'].dt.month
baskets['day'] = baskets['datetime'].dt.day
baskets['hour'] = baskets['datetime'].dt.hour
baskets['weekday'] = baskets['datetime'].dt.weekday

In [8]:
baskets.head(5)

,id,order_id,placed_at,merchant_id,sku_id,top_cat_id,sub_cat_id,qty,price,datetime,date,year,month,day,hour,weekday
0,126,23,2021-05-05 11:04:46.579,10,341,3.0,47.0,100,0.0,2021-05-05 11:04:46.579,2021-05-05,2021,5,5,11,2
1,166,41,2021-05-06 10:45:02.448,196,341,3.0,47.0,2,0.0,2021-05-06 10:45:02.448,2021-05-06,2021,5,6,10,3
2,167,42,2021-05-06 10:45:04.850,196,341,3.0,47.0,2,0.0,2021-05-06 10:45:04.850,2021-05-06,2021,5,6,10,3
3,168,43,2021-05-06 10:48:00.927,197,457,12.0,30.0,20,0.0,2021-05-06 10:48:00.927,2021-05-06,2021,5,6,10,3
4,169,44,2021-05-06 10:51:28.980,198,341,3.0,47.0,5,0.0,2021-05-06 10:51:28.980,2021-05-06,2021,5,6,10,3


#### Try to make algorithm that plots all purchases over months for random SKU (year doesn't matter right now)

/var/folders/_s/hz4hgk2d4hjckw16_s2z7k600000gn/T/ipykernel_7726/845837278.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  baskets[baskets['sku_id']== 341].sum().groupby(by=["sku_id"]).sum().plot()


ValueError: Grouper for 'sku_id' not 1-dimensional